I have at my disposal the box score of every nba game since 2003. Planning to predict the winner, spread, and over under .


In [1]:
#import dependences

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from sklearn import preprocessing, cross_validation
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense
from pandas import get_dummies

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def make_network(FILENAME,sklearn=False,keras=False,normalize=True,overunder=False,spread=False,moneyline=False):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    from sklearn.neural_network import MLPClassifier
    
    """
    Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and hot encoding is done. Other stuff too probably. Such as normalization, but I 
    didn't do that!
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Along with all the options as bools (not properly documented yet sorry)
    
    
    Returns
    -------
    
    
    model : object
        MLP which can predict the outcome of NBA games
        
    
    """
    
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    #*retrospectively that doesn't make sense, could be worth changing!
    data = read_csv(FILENAME) 
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values) 
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['Unnamed: 0'],data['GAME_ID'],data['Date'],data['Home'],data['Away'],data['PLUS_MINUS'],data['TOTAL']
    del data['FT'],data['FTA']
    data = get_dummies(data)

    #print(data)
    

    dat = []
    
    #reshape the dataset so now each colummn has roadstats and homestats concatenated into the same row, used for NN 
    
    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
    
    #convert list to array, now possible to array operations previously not possible
    dat = np.array(dat)   
    
    openingspreadS = dat[:,8] #what the predicted spread of ther game was. 
    roadpts = dat[:,7]       #column of all the points scored by road team 
    homepts = dat[:,52]
    endspreadS = roadpts-homepts  #all the final spreads of the game
            #concatenating all the arrays, looks messy but explanation doen in another nb. 
    x1 = dat[:,0:7] #road offensive rebounds to blocks
    x2 = dat[:,9:42] # road 3p% to team name (hot encoded)

    x3 = dat[:,45:52] #home offensive rebounds to blocks
    x4  =  dat[:,54:87] #home 3p% to hot encoded team name   
                      
    x5 = dat[:,8]              
    X1 = np.concatenate((x1,x2),axis=1)
    X2 = np.concatenate((x3,x4),axis=1)
    X3 = np.concatenate((X1,X2),axis=1)
    
    y = []
    
    if overunder:
        X = X3
        #[OVER,PUSH,UNDER]
        y = dat[:,42:45]
        #save outcomes for all, easy to show over under. 

    
    if spread:
        #include initial spread of the game. 
        X = np.column_stack((X3,x5))

        for j in range(len(endspreadS)):  
            openspread = openingspreadS[j]
       # print("this is the spread of the road team " + str(openspread))
            endspread = endspreadS[j]
       # print("the road team won by  .. " + str(endspread))
       # if endspread>openspread:
        #    y.append(np.array([0,1,0]))  #OK, now make sure this is formateed properly!
            if openspread + endspread <0:
                y.append(np.array([0,1,0]))  #home team covered
            elif openspread + endspread >0:
                y.append(np.array([1,0,0]))  #road covered
            else: 
                y.append(np.array([0,0,1]))  #push!

    
    if moneyline:
        X = X3
        for j in range(len(endspreadS)):  
            if endspreadS[j]<0:
                #means the home team had more points
                y.append(np.array([0,1]))
            else:
                y.append(np.array([1,0])) #alternatively, a road team victory. 
          

    #Now I iterated over all these, and hot encoded the labels of each to see whether or not the spread was covered
    #and by what team. 


        
    y = np.array(y)  #same explanation as above
                         
    #since everything got out of order I have to mash it together myself. 
    if normalize:
        
        scaler = MinMaxScaler()
        MinMaxScaler(copy=True, feature_range=(0, 1))

        scaler.fit(X)
        X = scaler.transform(X)
    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.27)
    #print((X[0]))
    #print(np.shape(X[0]))

    #now to construct a model 
    if sklearn: 
        model = MLPClassifier()
        model.shuffle = True
        model.batch_size = 25
    #model.n_layers_ = 1000000
    #model.n_outputs_= 1000000
    #These don't do anything, have to adjust the layers in some different way! Keras is useful for this.
        model.fit(X_train,y_train)
        print(model.score(X_test,y_test))
    if keras:
        print("keras NN goes here")
        model=Sequential()
        model.add(Dense(22,input_dim=np.shape(X)[1],activation='relu'))
        model.add(Dense(30,activation='relu'))
        model.add(Dense(50,activation='relu'))
        model.add(Dense(30,activation='relu'))
        model.add(Dense(22,activation='relu'))

        model.add(Dense(3,activation='sigmoid'))

        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        model.fit(X_train,y_train,batch_size=40,epochs=20,validation_split=.2)
        scores = model.evaluate(X_test,y_test)
        print(scores[1])

    
    
    return model,scaler
    

In [ ]:
nbapredictorOU,scaler = make_network('NBADATA.csv',sklearn=True,keras=False,normalize=True,overunder=True)

In [3]:
nbapredictorSPREAD,scaler = make_network('NBADATA.csv',sklearn=True,keras=False,normalize=True,spread=True)

0.831918505942275


In [ ]:
nbapredictorML,scaler = make_network('NBADATA.csv',sklearn=True,keras=False,normalize=True,moneyline=True)

The next function I have is for turning the current nba team statistics (either over the entire season or some stretch) into an array of the same shape and information as the one used for the box scores above. 

In [16]:
def get_splits(TEAMABR,ten=True,twenty=False,regseason=False):
    """returns the splits of a team over the past N days. Will consider changing this to a from - to thing for different dates. 
    
    Designated splits are available, with more to come such as H2H against a certain team
    
    """
    
    import numpy as np
    from nba_py import team
    teams = team.TeamList()
    teamids = teams.info()
    teamids = teamids[:-15]
    teamids = teamids[['TEAM_ID','ABBREVIATION']]

    teamids = teamids.rename(index=str, columns={"ABBREVIATION": "Team"})
    teamids = teamids.replace('BKN','BRK')
    teamids = teamids.sort_values('Team')

    TEAM_ID = teamids.loc[teamids['Team'] == TEAMABR].values[0,0]
    teamids = pd.get_dummies(teamids)
    teamarray = teamids.loc[teamids['TEAM_ID'] == TEAM_ID].values[0,1:]


    TEAM = team.TeamLastNGamesSplits(team_id=TEAM_ID)
    if ten:
        df = TEAM.last10()
    if twenty:
        df = TEAM.last20()
    if regseason:
        TEAM = team.TeamGeneralSplits(team_id=TEAM_ID)
        df = TEAM.overall()
   # if five:
        #df = TEAM
    
    df = df[['OREB','DREB','REB','PF','STL','TOV','BLK','FG3_PCT','FG_PCT','FT_PCT']].values
    
    teamsplits = np.concatenate((df[0],teamarray))
    return teamsplits



In [17]:
bos = get_splits('BOS')
cle = get_splits('CLE')
hou = get_splits('HOU')
gsw = get_splits('GSW')

In [18]:
def spread_game_maker(roadteam,hometeam,spread,scaler):
    import numpy as np

    """This includes the option to include the spread (OF THE ROAD TEAM), which isn't the case for the other ones. 
    
        After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    spread = np.array([spread])
    game = np.concatenate((roadteam,hometeam))
    game = np.concatenate((game,spread))
    game = [game]
    game = scaler.transform(game)
    return game

In [19]:
#Conference Finals

clebos1 = spread_game_maker(cle,bos,-1,scaler)
clebos2 = spread_game_maker(cle,bos,6,scaler)
clebos3 = spread_game_maker(bos,cle,7,scaler)
clebos4 = spread_game_maker(bos,cle,6.5,scaler)
#clebos3

gswhou1 = spread_game_maker(gsw,hou,1.5,scaler)
gswhou2 = spread_game_maker(gsw,hou,2,scaler)
gswhou3 = spread_game_maker(hou,gsw,7.5,scaler)
gswhou4 = spread_game_maker(hou,gsw,8,scaler)



In [23]:
nbapredictorSPREAD.predict(clebos4)

array([[1, 0, 0]])

In [ ]:
ind = get_splits('IND')
cle = get_splits('CLE')
was = get_splits('WAS')
tor = get_splits('TOR')
mia = get_splits('MIA')
phi = get_splits('PHI')
mil = get_splits('MIL')
bos = get_splits('BOS')

In [ ]:
minn = get_splits('MIN')
hou = get_splits('HOU')
uta = get_splits('UTA')
okc = get_splits('OKC')
nop = get_splits('NOP')
por = get_splits('POR')
sas = get_splits('SAS')
gsw = get_splits('GSW')

In [ ]:
indcle1 = spread_game_maker(ind,cle,7.5,scaler)
indcle2 = spread_game_maker(ind,cle,8,scaler)
cleind3 = spread_game_maker(cle,ind,1.5,scaler)
cleind4 = spread_game_maker(cle,ind,-1.5,scaler)
indcle5 = spread_game_maker(ind,cle,6.5,scaler)
cleind6 = spread_game_maker(cle,ind,-1.5,scaler)
indcle7 = spread_game_maker(ind,cle,5,scaler)


wastor1 = spread_game_maker(was,tor,7.5,scaler)
wastor2 = spread_game_maker(was,tor,7,scaler)
wastor3 = spread_game_maker(tor,was,1,scaler)
wastor4 = spread_game_maker(tor,was,-1.5,scaler)
wastor5 = spread_game_maker(was,tor,7,scaler)
wastor6 = spread_game_maker(tor,was,-2,scaler)

miaphi1 = spread_game_maker(mia,phi,6,scaler)
miaphi2 = spread_game_maker(mia,phi,6.5,scaler)
miaphi3 = spread_game_maker(phi,mia,-2.5,scaler)
miaphi4 = spread_game_maker(phi,mia,-4,scaler)
miaphi5 = spread_game_maker(mia,phi,10,scaler)

milbos1 = spread_game_maker(mil,bos,4,scaler)
milbos2 = spread_game_maker(mil,bos,13,scaler)
milbos3 = spread_game_maker(bos,mil,4.5,scaler)
milbos4 = spread_game_maker(bos,mil,6,scaler)
milbos5 = spread_game_maker(mil,bos,4.5,scaler)
milbos6 = spread_game_maker(bos,mil,4.5,scaler)
milbos7 = spread_game_maker(mil,bos,5,scaler)

In [ ]:
#western conference round 1 matchups

minhou1 = spread_game_maker(minn,hou,11.5,scaler)
minhou2 = spread_game_maker(minn,hou,10.5,scaler)
minhou3 = spread_game_maker(hou,minn,-6,scaler)
minhou4 = spread_game_maker(hou,minn,-6,scaler)
minhou5 = spread_game_maker(minn,hou,12,scaler)

utaokc1 = spread_game_maker(uta,okc,4,scaler)
utaokc2 = spread_game_maker(uta,okc,3.5,scaler)
utaokc3 = spread_game_maker(okc,uta,5,scaler)
utaokc4 = spread_game_maker(okc,uta,4.5,scaler)
utaokc5 = spread_game_maker(uta,okc,2.5,scaler)
utaokc6 = spread_game_maker(okc,uta,7,scaler)

noppor1 = spread_game_maker(nop,por,5,scaler)
noppor2 = spread_game_maker(nop,por,6,scaler)
noppor3 = spread_game_maker(por,nop,4,scaler)
noppor4 = spread_game_maker(por,nop,7,scaler)


sasgsw1 = spread_game_maker(sas,gsw,8,scaler)
sasgsw2 = spread_game_maker(sas,gsw,9,scaler)
sasgsw3 = spread_game_maker(gsw,sas,-3.5,scaler)
sasgsw4 = spread_game_maker(gsw,sas,-5.5,scaler)
sasgsw5 = spread_game_maker(sas,gsw,11,scaler)



In [ ]:
#Round 2 Matchups

cletor1 = spread_game_maker(cle,tor,7,scaler)
cletor2 = spread_game_maker(cle,tor,6.5,scaler)
cletor3 = spread_game_maker(tor,cle,4.5,scaler)
cletor4 = spread_game_maker(tor,cle,5,scaler)

phibos1 = spread_game_maker(phi,bos,-5,scaler)
phibos2 = spread_game_maker(phi,bos,-3.5,scaler)
phibos3 = spread_game_maker(bos,phi,1.5,scaler)
phibos4 = spread_game_maker(bos,phi,7,scaler)
phibos5 = spread_game_maker(phi,bos,1.5,scaler)


utahou1 =  spread_game_maker(uta,hou,11.5,scaler)
utahou2 =  spread_game_maker(uta,hou,11,scaler)
utahou3 =  spread_game_maker(hou,uta,-4.5,scaler)
utahou4 =  spread_game_maker(hou,uta,-5.5,scaler)
utahou5 =  spread_game_maker(uta,hou,11.5,scaler)


nopgsw1 = spread_game_maker(nop,gsw,7,scaler)
nopgsw2 = spread_game_maker(nop,gsw,11,scaler)
nopgsw3 = spread_game_maker(gsw,nop,-4.5,scaler)
nopgsw4 = spread_game_maker(gsw,nop,-5.5,scaler)
nopgsw5 = spread_game_maker(nop,gsw,12,scaler)

In [ ]:
nbapredictorSPREAD.predict(clebos2)

x x y

In [ ]:
23+12

In [ ]:
35-29

In [ ]:
nbapredictorSPREAD.predict(clebos)

In [ ]:
def OU_ML_game_maker(roadteam,hometeam,scaler):
    import numpy as np

    """
        After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game = np.concatenate((roadteam,hometeam))
    game.append(spread)
    game = [game]
    game = scaler.transform(game)
    return game

In [ ]:
#clebos = game_maker(cle,bos,scaler)
gswhou = spread_game_maker(gsw,hou,2,scaler)

In [ ]:
g

In [ ]:
nbapredictorOU.predict(gswhou)

In [ ]:
nbapredictorML.predict(gswhou)

In [ ]:
nbapredictorSPREAD.predict(gswhou)

^ This is differnet from the original prediction, which said it was going to be Golden State. 

In [ ]:
nbapredictorOU.predict(clebos)

In [ ]:
nbapredictorML.predict(clebos)